# LIVRIA : Système de recommandation par filtrage colllaboratif (memory based method)



   Ce jupyter notebook contient le code permettant de prédire les livres susceptibles de plaire à l'utilisateur.

   Nous utiliserons dans un premier temps notre base de données issue d'un questionnaire dont vous pouvez observer les résultats dans le fichier answerLivria.csv. Ce dataset est axé sur les thèmes de prédilection des utilisateurs. Nous avons déjà nettoyé et vectorisé ce set de donnée (cf. dataCleaning.ipynb) et analysé les réponses (cf. dataVizualisation.ipynb).
   Dans un second temps, nous utiliserons la base de données Goodbooks-10k, qui a l'avantage de réunir beaucoup plus de réponses et qui comprend notamment les notes attribuées par les utilisateurs aux livres qu'ils ont lus.
   
* Le set de données de Goodbooks-10 :
http://fastml.com/goodbooks-10k-a-new-dataset-for-book-recommendations/

Vous pourrez retrouver le notebook jupyter dédié au collaborative filtering pour les thèmes de livres dans le fichier Livria_Themes_Collaborative_filtering.ipynb

De plus, nous allons mettre en place du filtrage collaboratif par la suite pour la prédiction (cf. notebook Goodbooks10k_Collaborative_filtering), et il serait sûrement plus judicieux comme choix de garder uniquement les utilisateurs qui ont noté au moins 5 livres.
Maintenant, gardons uniquement les utilisateurs qui ont noté au moins 5 livres. 

# I. Filtrage collaboratif pour les thèmes

## Import des librairies

On commence par importer les librairies utilisées dans ce notebook.

In [1]:
import numpy as np
import pandas as pd
import graphlab as gl

from sklearn.model_selection import train_test_split

# Plot data
import matplotlib.pyplot as plt
%matplotlib inline

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1555538078.log


This non-commercial license of GraphLab Create for academic use is assigned to yghennam@ensc.fr and will expire on April 16, 2020.


## Import des données

Vous pouvez retrouver l'ensemble des données utilisées dans le dossier './data'


In [2]:
# Lecture du fichier 'df_sortie.csv' contenant les thèmes choisis  et vectorisés des réponses du questionnaire.
dataNotes = pd.read_csv('./data/ratings.csv')

In [3]:
# On montre les premières lignes de dataTheme
dataNotes.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [4]:
print('Les dimensions de dataNotes sont de : ' + str(dataNotes.shape))

Les dimensions de dataNotes sont de : (5976479, 3)


Au vu des dimensions du tableau, il semble que le nombre de note, soit un peu moins de 6 millions, correspond avec la description faite du dataset sur le site dont vous pouvez retrouver le lien plus haut (cf. Introduction).

A priori, il doit y avoir 10 000 livres notés par 53 424 utilisateurs. Vérifions-le maintenant :

In [5]:
n_users = dataNotes['user_id'].nunique()
n_items = dataNotes['book_id'].nunique()
print ("Nombre d'utilisateurs = " + str(n_users) + ' | Nombre de livres = ' + str(n_items))

Nombre d'utilisateurs = 53424 | Nombre de livres = 10000


## Création des sets d'entraînement et de test

On sépare le dataset en deux set distincts : un pour l'entraînement de notre modèle de prédiction et un pour tester ce modèle. On garde 25% des données pour le set de test.

In [6]:
train_data_notes, test_data_notes = train_test_split(dataNotes, test_size=0.25)

## Filtrage collaboratif

Nous allons prendre en considération deux modèle pour le filtrage collaboratif, le "memory-based" et le "model-based".

D'abord, on crée une matrice utilisateur-livre pour l'entraînement des données.

In [1]:
train_data_notes_matrix = gl.SArray(np.zeros((n_users,n_items)))
for line in train_data_notes.itertuples():
    train_data_notes_matrix[line[1]-1, line[2]-1] = line[3]

NameError: name 'gl' is not defined

On crée ensuite une matrice utilisateur-thème pour tester le modèle.

In [ ]:
test_data_notes_matrix = gl.SArray(np.zeros((n_users,n_items)))
for line in test_data_notes.itertuples():
    test_data_notes_matrix[line[1]-1, line[2]-1] = line[3]

### I.1 Filtrage collaboratif avec la méthode Memory-Based 

L'idée sous-jacente derrière le modèle dit "**memory-based**" est de calculer et d'utiliser les **similarités** entre utilisateurs et/ou items -ici les thèmes- et d'utiliser ces facteurs comme des "poids"  permettant la prédiction d'un thème, d'une note attribuée à un livre, ou autre. 

Nous allons tester les deux types de filtrage collaboratif:

* Item-Item 
* Utilisateur-Item 

Nous utilisons le coéfficient de similarité. Pour cela, nous importons la fonction "pairwise_distances" de sklearn. 

On calcule d'abord la similarité entre les utilisateurs.

In [ ]:
from sklearn.metrics import pairwise
user_similarity_notes = pairwise.cosine_similarity(train_data_notes_matrix)

In [ ]:
user_similarity_theme[:5, 0:5]

Calcul de la similarité entre les thèmes :

In [ ]:
item_similarity_theme = pairwise.cosine_similarity(train_data_theme_matrix.T)

In [ ]:
item_similarity_theme[:5, 0:5]

On définit une méthode pour réaliser les prédictions. 

In [ ]:
def predict(choices, similarity, kind='user'):
    
    sum_sim = np.array([np.abs(similarity).sum(axis=1)])
    sum_sim[sum_sim == 0] = 1    
    if kind == 'user':
        return similarity.dot(choices) / sum_sim.T
    elif kind == 'item':
        return choices.dot(similarity) / sum_sim

Cette méthode permet de prédire les thèmes susceptibles d'intéresser un utilisateur. Soit elle prend en considération les thèmes qui lui plaisent déjà, soit elle regarde les thèmes de prédilection d'autres utilisateurs ayant donné des réponses similaires.

In [ ]:
item_prediction_theme = predict(train_data_theme_matrix, item_similarity_theme, 'item')

In [ ]:
item_prediction_theme[0:5,0:3]

In [ ]:
user_prediction_theme = predict(train_data_theme_matrix, user_similarity_theme, 'user')
user_prediction_theme[0:5,0:3]

On mesure la performance du modèle avec le calcul de la RMSE (root-mean-square error), c'est-à-dire la racine carrée de l'erreur quadratique. Cette méthode compare les vraies réponses aux réponses prédites par notre modèle.

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, true_value):
    prediction = prediction.flatten()
    true_value = true_value.flatten()
    return sqrt(mean_squared_error(prediction, true_value))

RMSE pour la prédiction basée sur la comparaison entre les utilisateurs. 

In [ ]:
user_CF_RMSE_theme = rmse(user_prediction_theme, test_data_theme_matrix)
print('RMSE basée sur les utilisateurs : ', user_CF_RMSE_theme)

In [ ]:
item_CF_RMSE_theme = rmse(item_prediction_theme, test_data_theme_matrix)
print('RMSE basée sur les thèmes : ', item_CF_RMSE_theme)

### I.2 Filtrage collaboratif avec la méthode Model-based

La même logique développée dans la partie précédente (cf. I.1 memory-based collaborative filtering) peut être utilisée dans la méthode dite "model-based" : les similarités entre utilisateurs et/ou items peuvent être calculées et associées à un *modèle*, et on peut ensuite utiliser ce modèle pour faire nos prédictions. 

Le filtrage collaboratif dit "model-based" repose sur la factorisation de matrice. 

Nous allons utiliser un algorithme "SVD-based" permettant de réduire les dimensions de notre set de données et de guarder les caractéristiques principales, c'est-à-dire déterminantes de nos prédictions.

Check the sparsity of our dataset.

In [ ]:
sparsity_theme=round(1.0-len(dataTheme)/float(1279*14),3)
print('The sparsity level of dataTheme is ' +  str(sparsity_theme*100) + '%')

Decompose the train_data_theme_matrix using the SVD method.

In [ ]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_theme_matrix, k = 13)

Create the diagonale matrix.

In [ ]:
s_diag_matrix=np.diag(s)

Compute the rating predictions from the decomposition values.

In [ ]:
X_pred_theme = np.dot(np.dot(u, s_diag_matrix), vt)

Compute the model RMSE.

In [ ]:
for k in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]:
    u, s, vt = svds(train_data_theme_matrix, k = k)
    s_diag_matrix=np.diag(s)
    X_pred_theme = np.dot(np.dot(u, s_diag_matrix), vt)
    print('SVD-based CF RMSE (k={}): {}'.format(k, str(rmse(X_pred_theme, test_data_theme_matrix))))

# II. Filtrage collaboratif avec Goodbooks-10k

## Import des données 

In [ ]:
# Lecture du fichier 'BX-Book-Ratings.csv' contenant les notes attribuées par les utilisateurs aux livres.
dataNote = pd.read_csv('./data/ratings.csv')

In [ ]:
dataNote.head()

In [ ]:
dataNote.shape

Au vu des dimensions du tableau, il semble que le nombre de note, soit un peu moins de 6 millions, correspond avec la description faite du dataset sur le site dont vous pouvez retrouver le lien plus haut (cf. Introduction).

A priori, il doit y avoir 10 000 livres notés par 53 424 utilisateurs. Vérifions-le maintenant :

In [ ]:
n_users = dataNote['user_id'].nunique()
n_items = dataNote['book_id'].nunique()
print ("Nombre d'utilisateurs = " + str(n_users) + ' | Nombre de livres = ' + str(n_items))

## Création des sets d'entraînement et de test

In [ ]:
train_data_note, test_data_note = train_test_split(dataNote, test_size=0.25)

## Filtrage collaboratf 

D'abord, on crée une matrice utilisateur-thème pour l'entraînement des données.

In [ ]:
train_data_note_matrix = np.zeros((n_users, n_items))
for line in train_data_note.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]
print(train_data_matrix)